In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random

In [6]:
def get_song_urls_list(url):
    song_urls_list = []

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    for a in soup.find_all('a', {'rel': 'bookmark'}):
        song_urls_list.append(a['href'])
    return song_urls_list


def scrape_first_page(start_url, empty_df):
    
    start_urls_list = get_song_urls_list(start_url)
    
    for url in start_urls_list:
        
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        tags = soup.find_all(['td','p'])
        td_tags = soup.find_all(['td'])
        p_tags = soup.find_all(['p'])
        
        kor = pd.DataFrame()
        eng = pd.DataFrame()

        if (td_tags[-1].text=='N/A') | (td_tags[-1].text=='soon') | (len(tags)<25) | (p_tags[-3].text[0:6]!='Korean'):
            time.sleep(random.randint(5,8))
            continue
        else:
            c = len(tags) - 3
            d = c - 7
            e = int(d/3)
            a = e + 7
            b = e + a

            for i in range(a,b):
                kor_lines = tags[i]
                kor_lines_series = pd.Series(kor_lines.text.split('\n'))
                kor = pd.concat([kor, kor_lines_series])
                kor = kor[kor[0]!=""]
            kor = kor.rename(columns={0:'kor'}).reset_index().drop(columns='index')

            for i in range(b,c):
                eng_lines = tags[i]
                eng_lines_series = pd.Series(eng_lines.text.split('\n'))
                eng = pd.concat([eng, eng_lines_series])
                eng = eng[eng[0]!=""]
            eng = eng.rename(columns={0:'eng'}).reset_index().drop(columns='index')
            
            if eng.shape != kor.shape:
                continue
            else:
                concatted = pd.concat([eng, kor],axis=1)
                concatted = concatted.drop_duplicates()
                concatted = concatted.query("eng != kor")

                empty_df = pd.concat([empty_df,concatted])

                time.sleep(random.randint(6,9))
    return empty_df


def scrape_it(start_url, empty_df, end):
    
    z=1
    print(f'Starting page: {z}')
    df = scrape_first_page(start_url, empty_df)
    
    for i in range(136,end+1):
        
        z+=1
        print(f'Starting page: {z}')
        next_url = f'https://colorcodedlyrics.com/category/krn/page/{i}'

        urls_list = get_song_urls_list(next_url)
        if (i==100) | (i==200) | (i==300) | (i==400) | (i==500) | (i==600) | (i==700):
            df.to_csv('lyrics_save{}.csv'.format(i), sep='\t')
            df.to_csv('lyrics_save{}.txt'.format(i), sep='\t')
            print(f'saved up tp page: {i}')

        for url in urls_list:
            if url == 'https://colorcodedlyrics.com/2019/11/sulli-amp-hara':
                continue
            r = requests.get(url)
            soup = BeautifulSoup(r.text, 'html.parser')
            tags = soup.find_all(['td','p'])
            td_tags = soup.find_all(['td'])
            p_tags = soup.find_all(['p'])
            
            kor = pd.DataFrame()
            eng = pd.DataFrame()

            if (len(td_tags)<5) | (len(tags)<25) | (len(p_tags)<7):
                continue
            if (td_tags[-1].text=='N/A') | (td_tags[-1].text=='soon') | (td_tags[-1].text=='Coming Soon!') | (p_tags[-3].text[0:6]!='Korean'):
                time.sleep(random.randint(5,8))
                continue
            else:
                c = len(tags) - 3
                d = c - 7
                e = int(d/3)
                a = e + 7
                b = e + a

                for i in range(a,b):
                    kor_lines = tags[i]
                    kor_lines_series = pd.Series(kor_lines.text.split('\n'))
                    kor = pd.concat([kor, kor_lines_series])
                    kor = kor[kor[0]!=""]
                kor = kor.rename(columns={0:'kor'}).reset_index().drop(columns='index')

                for i in range(b,c):
                    eng_lines = tags[i]
                    eng_lines_series = pd.Series(eng_lines.text.split('\n'))
                    eng = pd.concat([eng, eng_lines_series])
                    eng = eng[eng[0]!=""]
                eng = eng.rename(columns={0:'eng'}).reset_index().drop(columns='index')

                if eng.shape != kor.shape:
                    continue

                concatted = pd.concat([eng, kor],axis=1)
                concatted = concatted.drop_duplicates()
                concatted = concatted.query("eng != kor")

                df = pd.concat([df,concatted])

                time.sleep(random.randint(5,8))

        
    return df

In [4]:
start_url = 'https://colorcodedlyrics.com/category/krn'

empty = pd.DataFrame(columns = ['eng','kor'])

df = scrape_it(start_url=start_url, empty_df=empty, end=136)

Starting page: 1
Starting page: 2
Starting page: 3


In [5]:
df

,eng,kor
1,Whose words should I be following?,어느 장단에 맞춰야 될지
2,When I take one step forward,한 발자국 떼면
3,My shadow grows one step bigger,한 발자국 커지는 shadow
4,"Waking up and opening my eyes, where am I?",잠에서 눈을 뜬 여긴 또 어디
5,Maybe Seoul or New York or Paris,어쩜 서울 또 New York or Paris
...,...,...
47,"Where are you headed, would you like to get on...",어디로 가죠? (Oh) 이 차에 훌쩍 탈래요? (탈래요?)
48,The summer has changed us a bit,여름은 우릴 (Oh) 좀 바꿔놨죠
49,"Wow, we’re already here, it’s the beach",와 벌써 왔어 바다야 (바다야)
52,Try changing up your day once,하룰 한번 비틀어 봐요 (Oh oh)


In [7]:
url = 'https://colorcodedlyrics.com/2019/01/shaun-syon-lyrics-index'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
tags = soup.find_all(['td','p'])
td_tags = soup.find_all(['td'])
p_tags = soup.find_all(['p'])


In [12]:
len(tags)

14

In [15]:
len(td_tags)

12

In [14]:
len(p_tags)

2

In [16]:
df = pd.read_csv('lyrics_save100.csv', sep='\t')

In [17]:
df

,Unnamed: 0,eng,kor
0,1,Whose words should I be following?,어느 장단에 맞춰야 될지
1,2,When I take one step forward,한 발자국 떼면
2,3,My shadow grows one step bigger,한 발자국 커지는 shadow
3,4,"Waking up and opening my eyes, where am I?",잠에서 눈을 뜬 여긴 또 어디
4,5,Maybe Seoul or New York or Paris,어쩜 서울 또 New York or Paris
...,...,...,...
14537,42,The heart always acts like this,마음이란 게 원래 그래
14538,43,But none of those words comfort me,어떤 말이라도 위로가 안돼
14539,44,I’m alone in an empty room,홀로 텅 빈 방구석에
14540,45,Sitting in a corner,웅크리고 앉아 기대
